## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

c:\users\admin\anaconda3\envs\opencv-env\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
c:\users\admin\anaconda3\envs\opencv-env\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
c:\users\admin\anaconda3\envs\opencv-env\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
c:\users\admin\anaconda3\envs\opencv-env\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


[skearn datasets](https://scikit-learn.org/stable/datasets/index.html)

In [11]:
# 匯入資料集
diabetes = datasets.load_diabetes()

# 將資料分為訓練集 / 測試集
x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.3, random_state=50)

# 建立模型
regr = GradientBoostingRegressor()

In [12]:
# 訓練模型
regr.fit(x_train,y_train)

#把測試資料丟進模型做預測
y_pred = regr.predict(x_tets)

In [13]:
print(metrics.mean_squared_error(y_test, y_pred))

3510.5034476144915


In [14]:
# 設定要訓練的超參數組合
n_estimators = [100, 150, 200, 250, 300]
max_depth = [2, 3, 5, 7]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(regr, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    2.2s finished


In [15]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -3480.254872 using {'max_depth': 2, 'n_estimators': 100}


In [16]:
grid_result.best_params_

{'max_depth': 2, 'n_estimators': 100}

In [17]:
# 使用最佳參數重新建立模型
regr_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
regr_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = regr_bestparam.predict(x_test)

In [18]:
# 調整參數後約可降至 8.30 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

3474.6683637054443


## [ 常見問題 ]

Q：超參數調整對最終結果影響很⼤嗎？
<br>A：超參數調整通常都是機器學習專案的最後步驟，因為這對於最終的結果影響不會太多，多半是近⼀步提升 3-5 % 的準確率，但是好的特徵⼯程與資料清理是能夠⼀⼝氣提升 10-20 ％ 的準確率！因此建議專案⼀開始時，不需要花太多時間進⾏超參數的調整

## [ 延伸資料 ]

- 劍橋實驗室教你如何調參數  -  [How to tune machine learning model](https://cambridgecoding.wordpress.com/2016/04/03/scanning-hyperspace-how-to-tune-machine-learning-models/)
- 教你使⽤ Python 調整隨機森林參數  -  [Hyperparameter Tuning the Random Forest in Python](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)